In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%matplotlib inline
import numpy as np 
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression 
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import metrics

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Mercari/p2_train.csv')

In [4]:
data.head()

,Unnamed: 0,train_id,name,item_condition_id,brand_name,price_x_x_x_x,shipping,item_description,category_main,category_sub1,category_sub2,brand_name_count,"('price', 'mean')_x","('price_y', 'mean')",shipping_count,"('price_x', 'mean')_x",category_main_count,"('price_x_y', 'mean')",category_sub1_count,"('price_x_x', 'mean')_x",category_sub2_count,"('price_x_x_y', 'mean')",word_count,stopword_count,ner,pos,avg_word_len,char_count,num_digit,char_count_without_space,cleaned_item_description,word_count_cleaned,neg,neu,pos.1,compound,word_count_cleaned_count,"('price_x_x_x', 'mean')_x",char_count_cleaned,char_count_cleaned_count,"('price_x_x_x_y', 'mean')",char_count_without_spaces,char_count_without_spaces_count,"('price_x_x_x_x', 'mean')"
0,0,0,MLB Cincinnati Reds T Shirt Size XL,3,uknown,10.0,1,No description yet,Men,Tops,T-shirts,8553,20.537180,26.092650,8948,21.731001,1258,35.191574,318,18.921384,229,17.471616,3,2,0,1,5.333333,18,0,16,description,1,0.0,1.000,0.000,0.0000,1651,21.959419,11,1260,22.176984,11,1328,22.088855
1,1,1,Razer BlackWidow Chroma Keyboard,3,Razer,52.0,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts,3,37.000000,26.092650,11008,29.821403,1610,32.832919,49,71.551020,13,50.153846,36,19,1,14,4.250000,188,0,153,keyboard greandition work like come box port t...,16,0.0,0.676,0.324,0.7717,457,30.367615,104,68,36.161765,89,80,37.150000
2,2,2,AVA-VIV Blouse,1,Target,10.0,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse,22,18.090909,26.136259,8948,21.731001,8895,28.358572,1385,17.555235,279,15.516129,29,18,2,17,3.310345,124,0,96,adorable hint lace key hole pale pink x x avai...,11,0.0,0.714,0.286,0.4939,746,25.438338,57,126,28.261905,47,173,25.930636
3,3,3,Leather Horse Statues,1,uknown,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents,8553,20.537180,26.136259,8948,21.731001,846,24.395981,318,22.235849,163,22.429448,32,15,0,17,4.437500,173,0,142,new tag leather horse retail rm stand foot hig...,17,0.0,0.829,0.171,0.5106,396,28.085859,96,83,28.759036,80,92,28.820652
4,4,4,24K GOLD plated rose,1,uknown,44.0,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces,8553,20.537180,26.136259,11008,29.821403,8895,28.358572,858,25.737762,276,24.449275,5,2,0,4,7.400000,41,0,37,complete certificate authenticity,3,0.0,1.000,0.000,0.0000,943,22.453871,33,173,21.069364,31,196,25.107143


In [5]:
data = data.rename(columns={"price_x_x_x_x": "price"})

In [6]:
y = np.log1p(data['price'])

In [7]:
data = data.iloc[: , 1:]

In [8]:
data.isnull().sum()

train_id                            0
name                                0
item_condition_id                   0
brand_name                          0
price                               0
shipping                            0
item_description                    0
category_main                       0
category_sub1                       0
category_sub2                       0
brand_name_count                    0
('price', 'mean')_x                 0
('price_y', 'mean')                 0
shipping_count                      0
('price_x', 'mean')_x               0
category_main_count                 0
('price_x_y', 'mean')               0
category_sub1_count                 0
('price_x_x', 'mean')_x             0
category_sub2_count                 0
('price_x_x_y', 'mean')             0
word_count                          0
stopword_count                      0
ner                                 0
pos                                 0
avg_word_len                        0
char_count  

In [9]:
# Fast Cleaning of Data
data['shipping'] = data['shipping'].astype(str)
data['item_condition_id'] = data['item_condition_id'].astype(str)
data['cleaned_item_description'] = data['cleaned_item_description'].fillna('None')

In [10]:
data = data.drop('item_description', axis=1)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.20)

In [12]:
print('X_train:',X_train.shape,'y_train:',y_train.shape)
print('X_test:',X_test.shape,'y_test:',y_test.shape)

X_train: (15964, 42) y_train: (15964,)
X_test: (3992, 42) y_test: (3992,)


#### bag of words

In [13]:
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

In [14]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

print("="*100)


vectorizer1 = CountVectorizer(min_df=10,ngram_range=(1,4), max_features=5000)
vectorizer1.fit(X_train['cleaned_item_description'].values) # fit has to happen only on train data

# we use the fitted CountVectorizer to convert the text to vector
X_train_cleaned_item_description_bow = vectorizer1.transform(X_train['cleaned_item_description'].values)
X_test_cleaned_item_description_bow = vectorizer1.transform(X_test['cleaned_item_description'].values)

print("After vectorizations")
print(X_train_cleaned_item_description_bow.shape, y_train.shape)
print(X_test_cleaned_item_description_bow.shape, y_test.shape)
print("="*100)

(15964, 42) (15964,)
(3992, 42) (3992,)
After vectorizations
(15964, 4766) (15964,)
(3992, 4766) (3992,)


In [15]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print("="*100)

vectorizer23 = CountVectorizer()
vectorizer23.fit(X_train['name'].values) # fit has to happen only on train data

# we use the fitted CountVectorizer to convert the text to vector
X_train_name_ohe_1 = vectorizer23.transform(X_train['name'].values)
X_test_name_ohe_1 = vectorizer23.transform(X_test['name'].values)

print("After vectorizations")
print(X_train_name_ohe_1.shape, y_train.shape)
print(X_test_name_ohe_1.shape, y_test.shape)
print(vectorizer23.get_feature_names())
print("="*100)

(15964, 42) (15964,)
(3992, 42) (3992,)
After vectorizations
(15964, 9974) (15964,)
(3992, 9974) (3992,)
['00', '000', '00ct', '00g', '00p', '01', '02', '020', '03', '034', '04', '05', '06', '07', '0oz', '10', '100', '1000', '1000pcs', '101', '102', '104', '108', '1080', '10c', '10ft', '10k', '10m', '10ml', '10pc', '10s', '10th', '10x', '10x13', '11', '110', '113', '114', '115', '11c', '11s', '11w', '12', '1200', '121', '122', '1234', '12brush', '12m', '12month', '12months', '12mth', '12oz', '12pcs', '12s', '12th', '12us', '12wide', '12x12', '12x15', '12x16', '13', '130', '1300mah', '13c', '14', '1444', '1456', '14ct', '14k', '14kt', '14oz', '14pc', '15', '150', '1531', '159', '15items', '16', '160', '16g', '16gb', '16oz', '16w', '17', '177', '17oz', '17pcs', '18', '182', '18g', '18inches', '18k', '18m', '18months', '18mos', '18mth', '18pc', '18w', '19', '1914p', '1929', '194', '1940s', '1958', '1960', '1960s', '1968', '1969', '1970', '1971', '1972', '1974', '1975', '1978', '1980s', '1

In [16]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print("="*100)

vectorizer23 = CountVectorizer()
vectorizer23.fit(X_train['brand_name'].values) # fit has to happen only on train data

# we use the fitted CountVectorizer to convert the text to vector
X_train_brand_name = vectorizer23.transform(X_train['brand_name'].values)
X_test_brand_name = vectorizer23.transform(X_test['brand_name'].values)

print("After vectorizations")
print(X_train_brand_name.shape, y_train.shape)
print(X_test_brand_name.shape, y_test.shape)
print(vectorizer23.get_feature_names())
print("="*100)

(15964, 42) (15964,)
(3992, 42) (3992,)
After vectorizations
(15964, 1173) (15964,)
(3992, 1173) (3992,)
['21', '44', '5th', '90', 'abbott', 'abercrombie', 'acacia', 'acer', 'adagio', 'aden', 'adidas', 'adrianna', 'adriano', 'advantage', 'aerie', 'aeropostale', 'aerosoles', 'affliction', 'ag', 'aid', 'air', 'aldo', 'alex', 'alexander', 'alfani', 'all', 'allsaints', 'almost', 'alo', 'alpinestars', 'alstyle', 'altec', 'amazon', 'amazonbasics', 'ambiance', 'ameda', 'american', 'amusement', 'anais', 'anastasia', 'and', 'and1', 'anderson', 'angels', 'angie', 'angry', 'ani', 'ann', 'annabelle', 'anne', 'anthropologie', 'antik', 'antonio', 'ape', 'api', 'apparel', 'apple', 'apt', 'aqueon', 'ardell', 'arden', 'ariat', 'arizona', 'armani', 'armour', 'art', 'as', 'ashley', 'asics', 'asos', 'assn', 'asus', 'athleta', 'athletic', 'athletica', 'aurum', 'australia', 'aveda', 'aveeno', 'avent', 'avenue', 'avery', 'avia', 'avon', 'axe', 'aéropostale', 'babolat', 'baby', 'babybjorn', 'badgley', 'bahama

In [17]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print("="*100)

vectorizer23 = CountVectorizer()
vectorizer23.fit(X_train['category_main'].values) # fit has to happen only on train data

# we use the fitted CountVectorizer to convert the text to vector
X_train_category_main = vectorizer23.transform(X_train['category_main'].values)
X_test_category_main = vectorizer23.transform(X_test['category_main'].values)

print("After vectorizations")
print(X_train_category_main.shape, y_train.shape)
print(X_test_category_main.shape, y_test.shape)
print(vectorizer23.get_feature_names())
print("="*100)

(15964, 42) (15964,)
(3992, 42) (3992,)
After vectorizations
(15964, 12) (15964,)
(3992, 12) (3992,)
['beauty', 'collectibles', 'electronics', 'handmade', 'home', 'kids', 'men', 'other', 'outdoors', 'sports', 'vintage', 'women']


In [18]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print("="*100)

vectorizer23 = CountVectorizer()
vectorizer23.fit(X_train['category_sub1'].values) # fit has to happen only on train data

# we use the fitted CountVectorizer to convert the text to vector
X_train_category_sub1 = vectorizer23.transform(X_train['category_sub1'].values)
X_test_category_sub1 = vectorizer23.transform(X_test['category_sub1'].values)

print("After vectorizations")
print(X_train_category_sub1.shape, y_train.shape)
print(X_test_category_sub1.shape, y_test.shape)
print(vectorizer23.get_feature_names())
print("="*100)

(15964, 42) (15964,)
(3992, 42) (3992,)
After vectorizations
(15964, 133) (15964,)
(3992, 133) (3992,)
['24', '2t', '5t', 'accessories', 'and', 'antique', 'apparel', 'appliances', 'art', 'artwork', 'athletic', 'audio', 'automotive', 'baby', 'bags', 'bath', 'bathing', 'bedding', 'blazers', 'blouses', 'body', 'book', 'books', 'boys', 'cameras', 'car', 'cards', 'care', 'cell', 'children', 'cleaning', 'clothing', 'coats', 'collectibles', 'computers', 'consoles', 'crochet', 'daily', 'decor', 'diapering', 'dining', 'dolls', 'dresses', 'décor', 'electronics', 'ephemera', 'exercise', 'fan', 'feeding', 'footwear', 'fragrance', 'furniture', 'games', 'gear', 'geekery', 'girls', 'glass', 'golf', 'goods', 'gps', 'hair', 'handbags', 'health', 'holidays', 'home', 'hoodies', 'housewares', 'instruments', 'items', 'jackets', 'jeans', 'jewelry', 'kids', 'kitchen', 'magazines', 'makeup', 'maternity', 'media', 'men', 'miniatures', 'mos', 'music', 'musical', 'needlecraft', 'nursery', 'office', 'organization

In [19]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print("="*100)

vectorizer23 = CountVectorizer()
vectorizer23.fit(X_train['category_sub2'].values) # fit has to happen only on train data

# we use the fitted CountVectorizer to convert the text to vector
X_train_category_sub2 = vectorizer23.transform(X_train['category_sub2'].values)
X_test_category_sub2 = vectorizer23.transform(X_test['category_sub2'].values)

print("After vectorizations")
print(X_train_category_sub2.shape, y_train.shape)
print(X_test_category_sub2.shape, y_test.shape)
print(vectorizer23.get_feature_names())
print("="*100)

(15964, 42) (15964,)
(3992, 42) (3992,)
After vectorizations
(15964, 637) (15964,)
(3992, 637) (3992,)
['100', '50', '75', 'above', 'accents', 'accessories', 'action', 'adapters', 'adventure', 'air', 'albums', 'all', 'and', 'animal', 'animals', 'animation', 'apparel', 'appliances', 'applicators', 'apron', 'area', 'art', 'arts', 'aspirators', 'asymmetrical', 'athletic', 'audio', 'baby', 'backpack', 'backpacks', 'bag', 'baggy', 'bags', 'bakeware', 'balls', 'baseball', 'basic', 'basketball', 'baskets', 'bath', 'bathing', 'bathroom', 'batteries', 'beach', 'bead', 'bed', 'bedding', 'bedroom', 'beer', 'belt', 'belts', 'beverage', 'bibles', 'bike', 'binoculars', 'bins', 'biographies', 'biography', 'blankets', 'blazer', 'blouse', 'blouses', 'blu', 'board', 'body', 'book', 'bookmark', 'books', 'booster', 'boot', 'boots', 'bottle', 'bottles', 'bottoms', 'bouncers', 'bowl', 'bowls', 'box', 'boxes', 'boyfriend', 'boys', 'bracelet', 'bracelets', 'bras', 'breastfeeding', 'briefcases', 'brooch', 'bru

#### Item condition id & Shipping

In [20]:

from scipy.sparse import csr_matrix

In [21]:
X_dummies = csr_matrix(pd.get_dummies(X_train[['item_condition_id', 'shipping']],
                                          sparse=True).values)

In [22]:
X_dummies_test = csr_matrix(pd.get_dummies(X_test[['item_condition_id', 'shipping']],
                                          sparse=True).values)

In [23]:
print(X_dummies.shape, y_train.shape)

print(X_dummies_test.shape, y_test.shape)

(15964, 7) (15964,)
(3992, 7) (3992,)


#### Numerical

In [24]:
data.dtypes

train_id                             int64
name                                object
item_condition_id                   object
brand_name                          object
price                              float64
shipping                            object
category_main                       object
category_sub1                       object
category_sub2                       object
brand_name_count                     int64
('price', 'mean')_x                float64
('price_y', 'mean')                float64
shipping_count                       int64
('price_x', 'mean')_x              float64
category_main_count                  int64
('price_x_y', 'mean')              float64
category_sub1_count                  int64
('price_x_x', 'mean')_x            float64
category_sub2_count                  int64
('price_x_x_y', 'mean')            float64
word_count                           int64
stopword_count                       int64
ner                                  int64
pos        

In [25]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()

normalizer.fit(X_train['brand_name_count'].values.reshape(-1,1))

#X_train_price_norm = normalizer.transform(X_train['price'].values.reshape(-1,1))
X_train_brand_name_count_norm = normalizer.transform(X_train['brand_name_count'].values.reshape(-1,1))

X_test_brand_name_count_norm = normalizer.transform(X_test['brand_name_count'].values.reshape(-1,1))

print("After normalizations")
print(X_train_brand_name_count_norm.shape, y_train.shape)

print(X_test_brand_name_count_norm.shape, y_test.shape)

After normalizations
(15964, 1) (15964,)
(3992, 1) (3992,)


In [26]:
X_train_norm = X_train.drop(['train_id','name','item_condition_id','brand_name','shipping','category_main','category_sub1','category_sub2','cleaned_item_description'], axis=1)
X_test_norm = X_test.drop(['train_id','name','item_condition_id','brand_name','shipping','category_main','category_sub1','category_sub2','cleaned_item_description'], axis=1)

In [27]:
# applying for loop to bring all the variables in range 0 to 1

for i in X_train_norm.columns[0:]:
    X_train_norm[i] = (X_train_norm[i] - X_train_norm[i].min()) / (X_train_norm[i].max() - X_train_norm[i].min())

In [28]:
# applying for loop to bring all the variables in range 0 to 1

for i in X_test_norm.columns[0:]:
    X_test_norm[i] = (X_test_norm[i] - X_test_norm[i].min()) / (X_test_norm[i].max() - X_test_norm[i].min())

In [29]:
X_train_n = X_train_norm.to_numpy()
X_test_n = X_test_norm.to_numpy()

In [30]:
print(X_train_n.shape, y_train.shape)

print(X_test_n.shape, y_test.shape)

(15964, 33) (15964,)
(3992, 33) (3992,)


In [31]:
# merge two sparse matrices: https://stackoverflow.com/a/19710648/4084039
from scipy.sparse import hstack

In [32]:
X_train = hstack((X_train_category_sub2, X_train_category_sub1, X_train_category_main, X_train_brand_name, X_dummies,X_train_n, X_train_name_ohe_1, X_train_cleaned_item_description_bow)).tocsr()

X_test = hstack((X_test_category_sub2, X_test_category_sub1, X_test_category_main, X_test_brand_name, X_dummies_test, X_test_n, X_test_name_ohe_1, X_test_cleaned_item_description_bow)).tocsr()

print("Final Data matrix")
print(X_train.shape, y_train.shape)

print(X_test.shape, y_test.shape)
print("="*100)

Final Data matrix
(15964, 16735) (15964,)
(3992, 16735) (3992,)


#### Ridge regression L2

In [33]:
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
#from sklearn import linear_model
from sklearn.model_selection import GridSearchCV

In [34]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
#from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
parameters = {"alpha":[0.00000001,0.00001,0.0001,0.001,0.01,0.1,0,1,10,100,1000,10000,100000]}
ridgeReg = Ridge(solver = "lsqr", fit_intercept=False)
lr_reg = GridSearchCV(ridgeReg,param_grid =parameters,n_jobs=-1)
lr_reg.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=False,
                             max_iter=None, normalize=False, random_state=None,
                             solver='lsqr', tol=0.001),
             iid='deprecated', n_jobs=-1,
             param_grid={'alpha': [1e-08, 1e-05, 0.0001, 0.001, 0.01, 0.1, 0, 1,
                                   10, 100, 1000, 10000, 100000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
lr_reg.best_params_

{'alpha': 10}

In [ ]:
ridgeReg = Ridge(alpha=10,solver = "lsqr", fit_intercept=False )
ridgeReg.fit(X_train, y_train)

Ridge(alpha=10, copy_X=True, fit_intercept=False, max_iter=None,
      normalize=False, random_state=None, solver='lsqr', tol=0.001)

In [ ]:
r_pred_train= ridgeReg.predict(X_train)
train_ridge_RMSLE = np.sqrt(mean_squared_error(y_train, r_pred_train))
print(' Fold %02d Ridge RMSLE: ', train_ridge_RMSLE)

 Fold %02d Ridge RMSLE:  0.26843747895524733


In [ ]:
r_y_pred = ridgeReg.predict(X_test)
val_ridge_RMSLE = np.sqrt(mean_squared_error(y_test, r_y_pred))
print(' Fold %02d Ridge RMSLE: ', val_ridge_RMSLE)

 Fold %02d Ridge RMSLE:  0.37027226203852703


#### Lasso regression L1

In [ ]:
from sklearn.linear_model import Lasso

parameters = {"alpha":[0.00000001,0.00001,0.0001,0.001,0.01,0.1,0,1,10,100,1000,10000,100000]}
lassoReg = Lasso(fit_intercept=True)
lasso_reg = GridSearchCV(lassoReg,param_grid =parameters,n_jobs=-1)
lasso_reg.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'alpha': [1e-08, 1e-05, 0.0001, 0.001, 0.01, 0.1, 0, 1,
                                   10, 100, 1000, 10000, 100000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
lasso_reg.best_params_

{'alpha': 0.0001}

In [ ]:
lassoReg = Lasso(alpha=0.0001, fit_intercept=True)
lassoReg.fit(X_train, y_train)

Lasso(alpha=0.0001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [ ]:
l_pred_train= lassoReg.predict(X_train)
train_lasso_RMSLE = np.sqrt(mean_squared_error(y_train, l_pred_train))
print(' Fold %02d Ridge RMSLE: ', train_lasso_RMSLE)

 Fold %02d Ridge RMSLE:  0.2993453601431051


In [ ]:
l_y_pred = lassoReg.predict(X_test)
val_lasso_RMSLE = np.sqrt(mean_squared_error(y_test, l_y_pred))
print(' Fold %02d Ridge RMSLE: ', val_lasso_RMSLE)

 Fold %02d Ridge RMSLE:  0.3495390592929135


#### Elastic Net

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression

In [ ]:
parameters = {"alpha":[0.01]}
elasticnetReg = ElasticNet(fit_intercept=True)
elastic_reg = GridSearchCV(elasticnetReg,param_grid =parameters,n_jobs=-1)
elastic_reg.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
                                  l1_ratio=0.5, max_iter=1000, normalize=False,
                                  positive=False, precompute=False,
                                  random_state=None, selection='cyclic',
                                  tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=-1, param_grid={'alpha': [0.01]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
elastic_reg.best_params_

{'alpha': 0.01}

In [ ]:
elasticnetReg = Lasso(alpha=0.01, fit_intercept=True)
elasticnetReg.fit(X_train, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [ ]:
e_pred_train= elasticnetReg.predict(X_train)
train_ela_RMSLE = np.sqrt(mean_squared_error(y_train, e_pred_train))
print(' Fold %02d Elastic RMSLE: ', train_ela_RMSLE)

 Fold %02d Elastic RMSLE:  0.4242693867022215


In [ ]:
e_y_pred = elasticnetReg.predict(X_test)
val_ela_RMSLE = np.sqrt(mean_squared_error(y_test, e_y_pred))
print(' Fold %02d Elastic RMSLE: ', val_ela_RMSLE)

 Fold %02d Elastic RMSLE:  0.4071392426214873


#### Light GBM

In [35]:
!pip install lightgbm

In [36]:
from lightgbm import LGBMRegressor
gridParams = {
    'learning_rate': [ 0.1,0.2,0.3,0.4,0.5],
    'n_estimators': [100,150, 200,250,300,400,500],
    'num_leaves': [20,30,63,80,100,120],
    'boosting_type' : ['gbdt'],
    'max_depth' : [2,3,4,5,6,7,8]
}
lgbm_params ={'subsample': 0.9, 'colsample_bytree': 0.8, 'min_child_samples': 50, 'objective': 'regression'
        }
lgbm_model = LGBMRegressor(**lgbm_params)
# Create the grid
grid = GridSearchCV(lgbm_model, gridParams, verbose=1, cv=3, n_jobs=-1)
# Run the grid
grid.fit(X_train, y_train,verbose=True)

print('Best parameters found by grid search are:', grid.best_params_)
print('Best score found by grid search is:', grid.best_score_)

Fitting 3 folds for each of 1470 candidates, totalling 4410 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   19.5s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 18.0min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 27.0min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 41.1min
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed: 52.7min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed: 67.0min
[Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed: 83.4min
[Parallel(n_jobs=-1)]: Done 4410 out of 4410 | elapsed: 92.5min finished


Best parameters found by grid search are: {'boosting_type': 'gbdt', 'learning_rate': 0.5, 'max_depth': 2, 'n_estimators': 500, 'num_leaves': 20}
Best score found by grid search is: 0.9995318641705673


In [37]:
print('Best parameters found by grid search are:', grid.best_params_)
print('Best score found by grid search is:', grid.best_score_)

Best parameters found by grid search are: {'boosting_type': 'gbdt', 'learning_rate': 0.5, 'max_depth': 2, 'n_estimators': 500, 'num_leaves': 20}
Best score found by grid search is: 0.9995318641705673


In [38]:
lgbm_params ={'subsample': 0.9, 'colsample_bytree': 0.8, 'min_child_samples': 50, 'objective': 'regression','boosting_type': 'gbdt','learning_rate': 0.5,'max_depth': 2,'n_estimators': 500,'num_leaves': 20,
      }
lgbm_model = LGBMRegressor(**lgbm_params)
lgbm_model.fit(X_train, y_train, verbose=True)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
              importance_type='split', learning_rate=0.5, max_depth=2,
              min_child_samples=50, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=500, n_jobs=-1, num_leaves=20,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=0.9,
              subsample_for_bin=200000, subsample_freq=0)

In [39]:
gbm_pred_train= lgbm_model.predict(X_train)
train_lgbm_RMSLE = np.sqrt(mean_squared_error(y_train, gbm_pred_train))
print(' Fold %02d Ridge RMSLE: ', train_lgbm_RMSLE)

 Fold %02d Ridge RMSLE:  0.007490772602593655


In [40]:
gbm_y_pred= lgbm_model.predict(X_test)
test_lgbm_RMSLE = np.sqrt(mean_squared_error(y_test, gbm_y_pred))
print(' Fold %02d Ridge RMSLE: ', test_lgbm_RMSLE)

 Fold %02d Ridge RMSLE:  0.01474696505738958


In [41]:
lgbm_model.feature_importances_

array([0, 0, 0, ..., 0, 0, 0])